In [1]:
import yaml
from jinja2 import Environment, PackageLoader, FileSystemLoader, StrictUndefined

In [2]:
import os

In [3]:
vars_file = os.path.join('.', 'hokusai', 'hokusai_vars.yml')

In [14]:
with open(vars_file, 'r') as f:
    struct = yaml.safe_load(f.read())
    print(struct)

{'vars': {'artsyNetWildcardSSLCert': 'arn:aws:iam::585031190124:server-certificate/2018-01-17_artsy-net-wildcard'}}


In [15]:
print(struct["vars"])

{'artsyNetWildcardSSLCert': 'arn:aws:iam::585031190124:server-certificate/2018-01-17_artsy-net-wildcard'}


In [16]:
template_file = os.path.join('.', 'hokusai', 'production.yml')

In [17]:
loader_kwargs = { "undefined": StrictUndefined }

In [18]:
env = Environment(loader=FileSystemLoader('./hokusai'), **loader_kwargs)

In [19]:
template_context = { "vars": struct["vars"] }

In [20]:
print(env.get_template('production.yml').render(**template_context))

---
apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: radiation-web
  namespace: default
spec:
  strategy:
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 0
    type: RollingUpdate
  template:
    metadata:
      labels:
        app: radiation
        component: web
        layer: application
      name: radiation-web
    spec:
      containers:
        - name: radiation-web
          env:
            - name: RAILS_SERVE_STATIC_FILES
              value: "true"
            - name: RAILS_LOG_TO_STDOUT
              value: "true"
            - name: PUMA_BIND
              value: "tcp://0.0.0.0:8080"
            - name: DD_TRACE_AGENT_HOSTNAME
              valueFrom:
                fieldRef:
                  fieldPath: spec.nodeName
          envFrom:
            - configMapRef:
                name: radiation-environment
          image: 585031190124.dkr.ecr.us-east-1.amazonaws.com/radiation:production
          imagePullPolicy: Always
          ports:
    